In [1]:
# check gcc version installed
!gcc --version

gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
!which python # should return /usr/local/bin/python

/usr/local/bin/python


In [3]:
!python --version

Python 3.10.12


# Installs and imports


---

In [1]:
!pip install pycuda # install cuda
!pip install -U git+https://github.com/arcadelab/deepdrr@dev # install deepdrr latest
!sudo apt-get install libgl1-mesa-dev xvfb # install headless display tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.4 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp310-cp310-linux_x86_64.whl size=661265 sha256=a137c2bcf417341e0c3385e9e7416f88d0b34ec3c452fd291d83cbe7d4f5f890
  Stored in directory: /root/.cache/pip/wheels/1d/7b/06/82a395a243fce00035dea9914d92bbef0013401497d849f8bc
Successfully built pycuda
  Cloning https://github.com/arcadelab/deepdrr (to revision dev) to /tmp/pip-req-build-t213tb7_
  Running command git clone --filter=blob:none --quiet https://github.com/arcadelab/deepdrr /tmp/pip-req-build-t213tb7_
  Running command git checkout -b dev --track origin/dev
  Switched to a new branch 'dev'
  Branch 'dev' set up

In [9]:
import deepdrr
import pycuda

import os, sys
import numpy as np
from pathlib import Path
import nibabel as nib
import nrrd

from PIL import Image
from pathlib import Path

from deepdrr import geo, Volume, MobileCArm, utils
from deepdrr.projector import Projector

from IPython.display import display


# Mount from Drive


---

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Upload Files


---


In [26]:
# from google.colab import files
# uploaded_files = files.upload()

# Read Files


---


In [29]:
"""
INPUT:
"""
VESSEL_CONTRAST = True # If vessels are contrasted
VOLUME_PATH_nrrd = "./drive/My Drive/Colab_Notebooks/D1.nrrd"
ARTERY_MASK_PATH_nrrd = "./drive/My Drive/Colab_Notebooks/D1.seg.nrrd"
VOLUME_PATH_nib = "./drive/My Drive/Colab_Notebooks/D1.nii.gz"
ARTERY_MASK_PATH_nib = "./drive/My Drive/Colab_Notebooks/D1_1.nii.gz"
# DRR_IMAGE_NAME = "./drive/MyDrive/Colab_Notebooks/sampleDRR.png"



In [30]:
# volume = Volume.from_nifti(Path(VOLUME_PATH), use_thresholding=True)
# DRR_IMAGE_NAME = "./drive/My Drive/Colab_Notebooks/2607_test_1.png"
# DRR_IMAGE_NAME_1 = "./drive/My Drive/Colab_Notebooks/2607_test_2.png"
# DRR_IMAGE_NAME_or =  "./drive/My Drive/Colab_Notebooks/"

In [40]:
# DRR_IMAGE_NAME = "./drive/MyDrive/Colab_Notebooks/2203_test_1.png"
# DRR_IMAGE_NAME_1 = "./drive/MyDrive/Colab_Notebooks/2203_test_2.png"
# DRR_IMAGE_NAME_or =  "./drive/MyDrive/Colab_Notebooks/"

# volume = Volume.from_nifti(Path(VOLUME_PATH), use_thresholding=True)
try:
  vol_with = Volume.from_nrrd(Path(VOLUME_PATH_nrrd), use_thresholding=True)
  vol_without = Volume.from_nrrd(Path(VOLUME_PATH_nrrd), use_thresholding=True)
  artery_mask, artery_mask_header = nrrd.read(Path(ARTERY_MASK_PATH_nrrd))
except:
  vol_with = Volume.from_nifti(Path(VOLUME_PATH_nib), use_thresholding=True)
  vol_without = Volume.from_nifti(Path(VOLUME_PATH_nib), use_thresholding=True)
  artery_mask, artery_mask_header = nib.load(Path(ARTERY_MASK_PATH_nib))

print(vol_with.shape)

(512, 666, 251)


# Hyperparameters


---



In [41]:
# Rotation angles of C-Arm:
ALPHA = -10
BETA = -10


In [43]:
    # C-Arm positioning!
    carm = MobileCArm(pixel_size=0.7, sensor_width=1440, sensor_height=1440,
                  source_to_isocenter_vertical_distance=742.5,
                  source_to_detector_distance=1259.65,
                  max_alpha=180,
                  min_alpha=-180)

    carm.reposition(vol_with.center_in_world)

    carm.move_to(isocenter= [ 0, 0, 0], alpha=0, beta=90, degrees=True)
    # carm.move_to(isocenter= [ 0, 0, 0], alpha=90, beta=90, degrees=True)

    param = 2.0

In [44]:
vol_with.materials["titanium"] = (1 == artery_mask).astype(np.float32)
vol_with.materials["bone"] = vol_with.materials["bone"] * (1.0 - vol_with.materials["titanium"])
vol_with.materials["soft tissue"] = vol_with.materials["soft tissue"] * (1.0 - vol_with.materials["titanium"])
vol_with.materials["bone"] = vol_with.materials["bone"] * param
vol_with.materials["soft tissue"] = vol_with.materials["soft tissue"] * 1.0
vol_with.materials["titanium"] = vol_with.materials["titanium"] * 1.0



vol_without.materials["titanium"] = (1 == artery_mask).astype(np.float32)
vol_without.materials["bone"] = vol_without.materials["bone"] * (1.0 - vol_without.materials["titanium"])
vol_without.materials["bone"] = vol_without.materials["bone"] * param
vol_without.materials["soft tissue"] = vol_without.materials["soft tissue"] + vol_without.materials["titanium"]
vol_without.materials["titanium"] = vol_without.materials["titanium"] * 0.0

# For one test image pair

---

In [49]:
with Projector(vol_with, carm=carm, neglog=True, spectrum="90KV_AL40") as projector:
    projection = projector()

with Projector(vol_without, carm=carm, neglog=True, spectrum="90KV_AL40") as projector:
    projection_2 = projector()

In [54]:
DRR_with_vessel = "./drive/MyDrive/Colab_Notebooks/DRR_with_test.png"
DRR_without_vessel = "./drive/MyDrive/Colab_Notebooks/DRR_without_test.png"

In [55]:
# image_gray1 = 1 - utils.neglog(projection)
image_gray1 = projection
# image_gray2 = 1 - utils.neglog(projection_2)
image_gray2 = projection_2

image_gray1 = (image_gray1 * 255).astype(np.uint8)
image_gray2 = (image_gray2 * 255).astype(np.uint8)

PIL_Image1 = Image.fromarray(image_gray1)
PIL_Image2 = Image.fromarray(image_gray2)

PIL_Image1.save(DRR_with_vessel)
PIL_Image2.save(DRR_without_vessel)
print("saved")

saved


# For more than one angle

---


In [56]:
outputs_path = "./drive/MyDrive/Colab_Notebooks"

In [57]:
  for i in range(90):

      carm.move_to(isocenter= [ 0, 0, 0], alpha=float(i*1.0), beta=90, degrees=True)


      with Projector(vol_with, carm=carm, neglog=True, spectrum="90KV_AL40") as projector:
          projection = projector()

      with Projector(vol_without, carm=carm, neglog=True, spectrum="90KV_AL40") as projector:
          projection_2 = projector()

      # image_gray1 = 1 - utils.neglog(projection)
      image_gray1 = projection
      # image_gray2 = 1 - utils.neglog(projection_2)
      image_gray2 = projection_2

      image_gray1 = (image_gray1 * 255).astype(np.uint8)
      image_gray2 = (image_gray2 * 255).astype(np.uint8)

      PIL_Image1 = Image.fromarray(image_gray1)
      PIL_Image2 = Image.fromarray(image_gray2)

      PIL_Image1.save(f'{outputs_path}/D1_with_for_{i}.png')
      PIL_Image2.save(f'{outputs_path}/D1_without_for_{i}.png')